<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 809ms/step - loss: 0.6748 - accuracy: 0.5803 - val_loss: 0.6635 - val_accuracy: 0.5610
Epoch 2/10
26/26 [==============================] - 20s 773ms/step - loss: 0.6399 - accuracy: 0.6436 - val_loss: 0.6299 - val_accuracy: 0.5902
Epoch 3/10
26/26 [==============================] - 20s 762ms/step - loss: 0.6119 - accuracy: 0.7032 - val_loss: 0.5661 - val_accuracy: 0.8341
Epoch 4/10
26/26 [==============================] - 20s 788ms/step - loss: 0.5641 - accuracy: 0.7506 - val_loss: 0.6400 - val_accuracy: 0.5659
Epoch 5/10
26/26 [==============================] - 20s 770ms/step - loss: 0.5298 - accuracy: 0.7567 - val_loss: 0.4491 - val_accuracy: 0.8244
Epoch 6/10
26/26 [==============================] - 20s 769ms/step - loss: 0.4540 - accuracy: 0.8163 - val_loss: 0.4016 - val_accuracy: 0.8585
Epoch 7/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3907 - accuracy: 0.8394 - val_loss: 0.7874 - val_accuracy: 0.5659

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.69.h5
26/26 - 21s - loss: 0.6839 - accuracy: 0.5523 - val_loss: 0.6861 - val_accuracy: 0.4634
Epoch 2/5

Epoch 00002: saving model to weights.02-0.72.h5
26/26 - 22s - loss: 0.6279 - accuracy: 0.6606 - val_loss: 0.7213 - val_accuracy: 0.4732
Epoch 3/5

Epoch 00003: saving model to weights.03-0.54.h5
26/26 - 21s - loss: 0.5756 - accuracy: 0.7092 - val_loss: 0.5400 - val_accuracy: 0.8146
Epoch 4/5

Epoch 00004: saving model to weights.04-0.48.h5
26/26 - 20s - loss: 0.5375 - accuracy: 0.7518 - val_loss: 0.4779 - val_accuracy: 0.8780
Epoch 5/5

Epoch 00005: saving model to weights.05-0.41.h5
26/26 - 20s - loss: 0.4841 - accuracy: 0.7883 - val_loss: 0.4102 - val_accuracy: 0.9317


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6815 - accuracy: 0.5560 - val_loss: 0.6907 - val_accuracy: 0.4537


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6768 - accuracy: 0.5803 - val_loss: 0.6559 - val_accuracy: 0.7024
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6265 - accuracy: 0.7007 - val_loss: 0.6017 - val_accuracy: 0.7171


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6598 - accuracy: 0.6083 - val_loss: 0.8736 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 21s - loss: 0.6137 - accuracy: 0.6946 - val_loss: 0.5701 - val_accuracy: 0.6927
Epoch 3/50
26/26 - 20s - loss: 0.5598 - accuracy: 0.7263 - val_loss: 0.5696 - val_accuracy: 0.6878
Epoch 4/50
26/26 - 20s - loss: 0.4852 - accuracy: 0.7956 - val_loss: 0.5550 - val_accuracy: 0.6976
Epoch 5/50
26/26 - 20s - loss: 0.4590 - accuracy: 0.7993 - val_loss: 0.4439 - val_accuracy: 0.8293
Epoch 6/50
26/26 - 20s - loss: 0.3678 - accuracy: 0.8577 - val_loss: 0.4558 - val_accuracy: 0.7951
Epoch 7/50
26/26 - 20s - loss: 0.3457 - accuracy: 0.8637 - val_loss: 0.3865 - val_accuracy: 0.8390
Epoch 8/50
26/26 - 20s - loss: 0.3263 - accuracy: 0.8747 - val_loss: 0.2810 - val_accuracy: 0.9024
Epoch 9/50
26/26 - 20s - loss: 0.2479 - accuracy: 0.9209 - val_loss: 0.2045 - val_accuracy: 0.9317
Epoch 10/50
26/26 - 20s - loss: 0.2387 - accuracy: 0.9209 - val_loss: 0.1829 - val_accuracy: 0.9463
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 804ms/step - loss: 0.6428 - accuracy: 0.6655 - val_loss: 0.6544 - val_accuracy: 0.5171
Epoch 2/5
26/26 [==============================] - 21s 791ms/step - loss: 0.5887 - accuracy: 0.6910 - val_loss: 0.5486 - val_accuracy: 0.7659
Epoch 3/5
26/26 [==============================] - 21s 796ms/step - loss: 0.4965 - accuracy: 0.7676 - val_loss: 0.5241 - val_accuracy: 0.7854
Epoch 4/5
26/26 [==============================] - 21s 809ms/step - loss: 0.4367 - accuracy: 0.8054 - val_loss: 0.4040 - val_accuracy: 0.7951
Epoch 5/5
26/26 [==============================] - 20s 776ms/step - loss: 0.4175 - accuracy: 0.7981 - val_loss: 0.4280 - val_accuracy: 0.8341


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.665450,0.643243,0.517073,0.654377
1,1,0.690998,0.586933,0.765854,0.548588
2,2,0.767640,0.496364,0.785366,0.524083
3,3,0.805353,0.435346,0.795122,0.404020
4,4,0.798053,0.416832,0.834146,0.428003


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6696 - accuracy: 0.5815 - val_loss: 0.6794 - val_accuracy: 0.4585

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6506 - accuracy: 0.6265 - val_loss: 0.6658 - val_accuracy: 0.5073

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6394 - accuracy: 0.6521 - val_loss: 0.6532 - val_accuracy: 0.6390

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6330 - accuracy: 0.7080 - val_loss: 0.6502 - val_accuracy: 0.6488

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 785ms/step - loss: 0.6302

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 855ms/step - loss: 0.6788 - accuracy: 0.5280 - val_loss: 0.6886 - val_accuracy: 0.4341
Epoch 2/50
26/26 [==============================] - 21s 797ms/step - loss: 0.6582 - accuracy: 0.6192 - val_loss: 0.6484 - val_accuracy: 0.6537
Epoch 3/50
25/26 [===========================>..] - ETA: 0s - loss: 0.6289 - accuracy: 0.7138
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 779ms/step - loss: 0.6240 - accuracy: 0.7153 - val_loss: 0.7055 - val_accuracy: 0.4439
Epoch 4/50
26/26 [==============================] - 21s 789ms/step - loss: 0.5963 - accuracy: 0.6813 - val_loss: 0.5930 - val_accuracy: 0.8146
Epoch 5/50
26/26 [==============================] - 21s 801ms/step - loss: 0.5767 - accuracy: 0.8127 - val_loss: 0.5826 - val_accuracy: 0.8098
Epoch 6/50
26/26 [==============================] - 21s 791ms/step - loss: 0.5637 - accuracy: 0.7932 - val_loss: 0.5590 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4902), started 0:17:12 ago. (Use '!kill 4902' to kill it.)